<a href="https://colab.research.google.com/github/netmatze/mlmatze/blob/main/fintune_alpaca_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git
!pip install huggingface
!pip install huggingface_hub
!pip -q install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.9 MB/s eta 0:00:00
  Installing bu

In [ ]:
# Code adapted from https://github.com/tloen/alpaca-lora
import torch
from peft import PeftModel
import transformers

from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-30b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-30b-hf",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

model = PeftModel.from_pretrained(
    model,
    "baseten/alpaca-30b",
    torch_dtype=torch.float16
)

def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""


model.eval()


def evaluate(
        instruction,
        input=None,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=2048,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:2405 in from_pretrained    │
│                                                                                                  │
│   2402 │   │   # We'll need to download and cache each checkpoint shard if the checkpoint is sh  │
│   2403 │   │   if is_sharded:                                                                    │
│   2404 │   │   │   # rsolved_archive_file becomes a list of files that point to the different c  │
│ ❱ 2405 │   │   │   resolved_archive_file, sharded_metadata = get_checkpoint_shard_files(         │
│   2406 │   │   │   │   pretrained_model_name_or_path,                                            │
│   2407 │   │   │   │   resolved_archive_file,                                                    │
│   2408 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/utils/hub.py:919 in                          │
│ get_checkpoint_shard_files                                                                       │
│                                                                                                  │
│    916 │   for shard_filename in shard_filenames:                                                │
│    917 │   │   try:                                                                              │
│    918 │   │   │   # Load from URL                                                               │
│ ❱  919 │   │   │   cached_filename = cached_file(                                                │
│    920 │   │   │   │   pretrained_model_name_or_path,                                            │
│    921 │   │   │   │   shard_filename,                                                           │
│    922 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/utils/hub.py:409 in cached_file              │
│                                                                                                  │
│    406 │   user_agent = http_user_agent(user_agent)                                              │
│    407 │   try:                                                                                  │
│    408 │   │   # Load from URL or cache if already cached                                        │
│ ❱  409 │   │   resolved_file = hf_hub_download(                                                  │
│    410 │   │   │   path_or_repo_id,                                                              │
│    411 │   │   │   filename,                                                                     │
│    412 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn     │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                   

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

# model_path = "decapoda-research/llama-30b-hf"
# peft_path = 'serpdotai/llama-hh-lora-30B'
# tokenizer_path = 'decapoda-research/llama-30b-hf'

# "decapoda-research/llama-30b-hf"
# tokenizer = LlamaTokenizer.from_pretrained('decapoda-research/llama-30b-hf')
# model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-30b-hf",
#                                          load_in_8bit=True,
#                                          device_map="auto") # or something like {"": 0}
# model = PeftModel.from_pretrained(
#                                   model,
#                                   'serpdotai/llama-hh-lora-30B',
#                                   # torch_dtype=torch.float16,
#                                   device_map="auto") # or something like {"": 0}

# "decapoda-research/llama-13b-hf"
# tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-13b-hf")
# model = LLaMAForCausalLM.from_pretrained(
#         "decapoda-research/llama-13b-hf",
#         load_in_8bit=True,
#         # torch_dtype=torch.float16,
#         device_map="auto",
# )
# model = PeftModel.from_pretrained(
#     model, "baruga/alpaca-lora-13b",
#     # torch_dtype=torch.float16
# )

# "decapoda-research/llama-7b-hf"
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
# model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")
model = PeftModel.from_pretrained(model, "netmatze/alpaca7B-lora")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-3ltjncwdr69pv --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true'), PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=h

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:
generation_config = GenerationConfig(
    temperature=0.5,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=556
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

In [ ]:
evaluate('Instruction: write a python program that prints the first 10 Fibonacci numbers.')

Response: def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))


In [ ]:
evaluate('Instruction: write a python program that uses sklearn to classify the iris dataaset.')

Response: import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


In [ ]:
evaluate('Instruction: write a c# program that creates a linked list implementation.')

Response: public class LinkedList<T>
{
    private Node<T> head;
    private Node<T> tail;

    public LinkedList()
    {
        head = null;
        tail = null;
    }

    public void add(T item)
    {
        Node<T> newNode = new Node<T>(item);
        if (head == null)
        {
            head = tail = newNode;
        }
        else
        {
            tail.next = newNode;
            tail = newNode;
        }
    }

    public T remove()
    {
        if (head == null)
        {
            return null;
        }
        else
        {
            T item = head.item;
            head = head.next;
            if (head == null)
            {
                tail = null;
            }
            else
            {
                tail = head;
            }
            return item;
        }
    }

    public void print()
    {
        Node<T> current = head;
        while (current!= null)
        {
            Console.WriteLine(current.item);
            current = current.nex

In [ ]:
evaluate('Instruction: write a python program for object detection in an image with the cv2 libary.')

Response: import cv2
import numpy as np

img = cv2.imread('image.jpg', 0)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    area = cv2.contourArea(c)
    if area > 500:
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow("Object Detection", img)
cv2.waitKey(0)


In [ ]:
evaluate('Instruction: write a python program for object detection in an image and classification of the object with the cv2 libary.')

Response: import numpy as np
import matplotlib.pyplot as plt
import cv2

img = cv2.imread('image.jpg', 0)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    area = cv2.contourArea(c)
    if area > 500:
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

plt.imshow(img)
plt.show()


In [ ]:
#evaluate(input("Instruction: "))
evaluate('Instruction: cooking recipe with wild garlic, rice and peas')

Response: Wild Garlic, Rice and Peas Recipe: 

Ingredients: 
- 2 cloves of garlic, chopped 
- 1 tablespoon of olive oil 
- 1 cup of rice 
- 1 cup of frozen peas 
- 2 cups of vegetable stock 
- Salt and pepper to taste 

Instructions: 
1. Heat the olive oil in a large saucepan over medium heat. 
2. Add the garlic and cook for 1 minute. 
3. Add the rice and cook for 2 minutes. 
4. Add the peas and stock and bring to a boil. 
5. Reduce the heat and simmer for 15 minutes or until the rice is cooked. 
6. Season with salt and pepper to taste. 
7. Serve hot.


In [ ]:
evaluate('Translate: Ich koche auf meinem Herd eine gute Nudelsuppe')

Response: I am cooking a good noodle soup on my stove.


In [ ]:
evaluate('Rewrite: is it very good to go to the school in japan')

Response: It is highly recommended to attend school in Japan.


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# model.push_to_hub("netmatze/alpaca7B-lora", use_auth_token=True)